In [1]:
import requests
import lxml.html as lh
import pandas as pd
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize

C:\Users\Kanchan\Anaconda3\envs\eye-track\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [4]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

#Removing \n from the dataframe
df = df.replace('\n',' ', regex=True)
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [6]:
df.columns=['Postal Code','Borough','Neighbourhood']
#Grouping by postal codes and aggregating neighbourhoods
df=df.groupby("Postal Code").agg(lambda x:','.join(set(x)))

In [7]:
#Assigning not assigned neighbourhoods same as borough
df.loc[df['Neighbourhood'].isin( ["Not assigned",'Neighbourhood'])]=df.loc[df['Neighbourhood'].isin( ["Not assigned",'Borough'])]
    


In [8]:
#Ignoring not assigned boroughs
df = df.loc[df['Borough'] != 'Not assigned']
df


,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Rouge ,Malvern"
M1C,Scarborough,"Rouge Hill ,Highland Creek ,Port Union"
M1E,Scarborough,"Guildwood ,West Hill ,Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park ,East Birchmount Park ,Ionview"
M1L,Scarborough,"Golden Mile ,Oakridge ,Clairlea"
M1M,Scarborough,"Cliffside ,Scarborough Village West ,Cliffcrest"


In [9]:
#Resetting index
df.reset_index()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Rouge Hill ,Highland Creek ,Port Union"
2,M1E,Scarborough,"Guildwood ,West Hill ,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park ,East Birchmount Park ,Ionview"
7,M1L,Scarborough,"Golden Mile ,Oakridge ,Clairlea"
8,M1M,Scarborough,"Cliffside ,Scarborough Village West ,Cliffcrest"
9,M1N,Scarborough,"Cliffside West ,Birch Cliff"


In [10]:
#Reading geospatial_coordinates csv into a dataframe
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [11]:
#Merging the two dataframes
dfinal = df.merge(df2, on="Postal Code", how = 'inner')
dfinal

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill ,Highland Creek ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,West Hill ,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park ,East Birchmount Park ,Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile ,Oakridge ,Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside ,Scarborough Village West ,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West ,Birch Cliff",43.692657,-79.264848


In [12]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(dfinal['Borough'].unique()),
        dfinal.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighbourhoods.


In [13]:
dfinal['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [14]:
# convert an address into latitude and longitude values

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for latitude, longitude, label in zip(dfinal['Latitude'], dfinal['Longitude'], dfinal['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
CLIENT_ID = 'clientid' # Foursquare ID was inserted and removed for security reasons
CLIENT_SECRET = 'clientsecret' #Foursquare Secret was inserted and removed for security reasons

VERSION = '20180605' # Foursquare API version



In [17]:
dfinal.loc[0, 'Neighbourhood']

'Rouge ,Malvern '

In [18]:
neighborhood_latitude = dfinal.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfinal.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfinal.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge ,Malvern  are 43.806686299999996, -79.19435340000001.


In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=JHV042UB1TCUG4QS0TMLXLALVNAUOT5A23GOSPVGN1AII3QE&client_secret=30QUMQ2JOJWPNZD2ZZVBCSAU4BSWSCX3DWP3PVB44E4EWQ3A&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1b8eba2b274a002cffb363'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=dfinal['Neighbourhood'],
                                   latitudes=dfinal['Latitude'],
                                   longitudes=dfinal['Longitude']
                                  )



Rouge ,Malvern 
Rouge Hill ,Highland Creek ,Port Union 
Guildwood ,West Hill ,Morningside 
Woburn 
Cedarbrae 
Scarborough Village 
Kennedy Park ,East Birchmount Park ,Ionview 
Golden Mile ,Oakridge ,Clairlea 
Cliffside ,Scarborough Village West ,Cliffcrest 
Cliffside West ,Birch Cliff 
Scarborough Town Centre ,Dorset Park ,Wexford Heights 
Maryvale ,Wexford 
Agincourt 
Sullivan ,Tam O'Shanter ,Clarks Corners 
Steeles East ,Milliken ,L'Amoreaux East ,Agincourt North 
L'Amoreaux West 
Upper Rouge 
Hillcrest Village 
Henry Farm ,Oriole ,Fairview 
Bayview Village 
York Mills ,Silver Hills 
Willowdale ,Newtonbrook 
Willowdale South 
York Mills West 
Willowdale West 
Parkwoods 
Don Mills North 
Don Mills South ,Flemingdon Park 
Bathurst Manor ,Downsview North ,Wilson Heights 
York University ,Northwood Park 
Downsview East ,CFB Toronto 
Downsview West 
Downsview Central 
Downsview Northwest 
Victoria Village 
Woodbine Gardens ,Parkview Hill 
Woodbine Heights 
The Beaches 
Leaside 
Thorncliff

In [25]:
print(dfinal.shape)
dfinal.head()

(103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill ,Highland Creek ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,West Hill ,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide ,Richmond ,King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Bathurst Manor ,Downsview North ,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Beaumond Heights ,Albion Gardens ,Humbergate ,South Steeles ,Thistletown ,Silverstone ,Jamestown ,Mount Olive",11,11,11,11,11,11
"Bedford Park ,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
Caledonia-Fairbanks,6,6,6,6,6,6


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 280 uniques categories.


In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_onehot.shape

(2259, 280)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide ,Richmond ,King",0.000000,0.01,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Agincourt,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bathurst Manor ,Downsview North ,Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Beaumond Heights ,Albion Gardens ,Humbergate ,...",0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bedford Park ,Lawrence Manor East",0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Business Reply Mail Processing Centre 969 East...,0.052632,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Caledonia-Fairbanks,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667
9,Canada Post Gateway Processing Centre,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide ,Richmond ,King ----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4                  Bar  0.04


----Agincourt ----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3  Chinese Restaurant  0.25
4         Yoga Studio  0.00


----Bathurst Manor ,Downsview North ,Wilson Heights ----
                       venue  freq
0                Coffee Shop  0.11
1  Middle Eastern Restaurant  0.06
2                 Restaurant  0.06
3           Sushi Restaurant  0.06
4         Frozen Yogurt Shop  0.06


----Bayview Village ----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4    Mobile Phone Shop  0.00


----Beaumond Heights ,Albion Gardens ,Humbergate ,South Steeles ,Thistletown ,Silverstone ,Jamestown ,Mount Olive ----
            

           venue  freq
0          Trail  0.25
1          Field  0.25
2   Tennis Court  0.25
3   Hockey Arena  0.25
4  Metro Station  0.00


----Islington ,West Deane Park ,Princess Gardens ,Martin Grove ,Cloverdale ----
                             venue  freq
0                             Bank   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Kennedy Park ,East Birchmount Park ,Ionview ----
                       venue  freq
0           Department Store  0.25
1             Discount Store  0.25
2                 Playground  0.25
3                Coffee Shop  0.25
4  Middle Eastern Restaurant  0.00


----Kensington Market ,Chinatown ,Grange Park ----
                           venue  freq
0                           Café  0.08
1  Vegetarian / Vegan Restaurant  0.06
2                            Bar  0.04
3            Dumpling Restaurant  0.04
4                    

                 venue  freq
0          Pizza Place   0.2
1             Pharmacy   0.1
2         Noodle House   0.1
3  Fried Chicken Joint   0.1
4                 Bank   0.1


----Summerhill West ,South Hill ,Forest Hill SE ,Rathnelly ,Deer Park ----
                 venue  freq
0          Coffee Shop  0.14
1                  Pub  0.14
2   Light Rail Station  0.07
3  Fried Chicken Joint  0.07
4     Sushi Restaurant  0.07


----The Beaches ----
                             venue  freq
0                Health Food Store   0.2
1                            Trail   0.2
2             Other Great Outdoors   0.2
3                              Pub   0.2
4  Molecular Gastronomy Restaurant   0.0


----The Beaches West ,India Bazaar ----
                venue  freq
0                Park  0.11
1   Fish & Chips Shop  0.06
2  Italian Restaurant  0.06
3         Pizza Place  0.06
4                 Pub  0.06


----The Danforth West ,Riverdale ----
                    venue  freq
0        Greek Restauran

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,Richmond ,King",Coffee Shop,Café,Steakhouse,American Restaurant,Bar,Thai Restaurant,Gym,Cosmetics Shop,Hotel,Restaurant
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Electronics Store,Eastern European Restaurant,Empanada Restaurant,Dumpling Restaurant,Drugstore,Department Store
2,"Bathurst Manor ,Downsview North ,Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Frozen Yogurt Shop,Sandwich Place,Bridal Shop,Fast Food Restaurant,Diner,Restaurant,Bank,Supermarket
3,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,"Beaumond Heights ,Albion Gardens ,Humbergate ,...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pharmacy,Pizza Place,Sandwich Place,Coffee Shop,Video Store


In [34]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering.
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0])

In [35]:
#neighborhoods_venues_sorted  = neighborhoods_venues_sorted .reset_index(level=0, drop=True).reset_index()
neighborhoods_venues_sorted.reset_index(level=-1, drop=True)



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,Richmond ,King",Coffee Shop,Café,Steakhouse,American Restaurant,Bar,Thai Restaurant,Gym,Cosmetics Shop,Hotel,Restaurant
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Electronics Store,Eastern European Restaurant,Empanada Restaurant,Dumpling Restaurant,Drugstore,Department Store
2,"Bathurst Manor ,Downsview North ,Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Frozen Yogurt Shop,Sandwich Place,Bridal Shop,Fast Food Restaurant,Diner,Restaurant,Bank,Supermarket
3,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,"Beaumond Heights ,Albion Gardens ,Humbergate ,...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pharmacy,Pizza Place,Sandwich Place,Coffee Shop,Video Store
5,"Bedford Park ,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Liquor Store,Butcher,Indian Restaurant,Ice Cream Shop,Café,Sushi Restaurant,Japanese Restaurant
6,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Italian Restaurant,Steakhouse,Café,Cheese Shop,Beer Bar,Farmers Market
7,Business Reply Mail Processing Centre 969 East...,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Recording Studio,Restaurant,Burrito Place,Brewery
8,Caledonia-Fairbanks,Park,Women's Store,Pharmacy,Fast Food Restaurant,Market,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Sandwich Place,Gym / Fitness Center,Burrito Place,Mediterranean Restaurant,Fried Chicken Joint,American Restaurant,Women's Store,Dim Sum Restaurant


In [38]:




# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = neighborhoods_venues_sorted
toronto_merged=pd.concat([toronto_merged,neighborhoods_venues_sorted], axis=0,sort=False)
toronto_merged.head() # check the last columns!

toronto_merged.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Woodbine Heights,Cosmetics Shop,Beer Store,Athletics & Sports,Curling Ice,Skating Rink,Bus Stop,Pharmacy,Park,Electronics Store,Eastern European Restaurant
96,"York Mills ,Silver Hills",Cafeteria,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Deli / Bodega
97,York Mills West,Park,Bank,Convenience Store,Bar,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
98,"York University ,Northwood Park",Falafel Restaurant,Bar,Miscellaneous Shop,Coffee Shop,Caribbean Restaurant,Massage Studio,Metro Station,Diner,Discount Store,Dog Run
99,"Yorkville ,North Midtown ,The Annex",Coffee Shop,Sandwich Place,Café,Pizza Place,Burger Joint,Jewish Restaurant,Indian Restaurant,Pub,BBQ Joint,Cosmetics Shop


In [263]:
print(latitude)
print(longitude)


43.706748299999994
-79.5940544


In [256]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    #label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [43.706748299999994, -79.5940544],
        radius=5,
        popup=label,
        color=[colors.rgb2hex(i) for i in colors_array],

        fill=True,
        fill_color=[colors.rgb2hex(i) for i in colors_array],

        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters